In [1]:
# importing libs
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda,Dense, GaussianNoise
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
import random as rn

In [2]:
# defining parameters
M = 16
#Mmod= 16
R = 1/2
k = int(np.log2(M))
n_channel = int(k/R)

print ('M:',M,'k:',k,'n_coded:',n_channel,'R:',R)

M: 16 k: 4 n_coded: 8 R: 0.5


In [3]:
#generating data of size N
N = 1000000
label = np.random.randint(M,size=N)

In [4]:
# creating one hot encoded vectors
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

In [5]:
data = np.array(data)
print (data.shape)

(1000000, 16)


In [6]:
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

7 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
5 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
10 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
14 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
10 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
7 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
8 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
12 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [7]:
lr=0.001
Eb_N0db=5
Eb_N0=10.0**(Eb_N0db/10.0)

In [8]:
# defining autoencoder and it's layer
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='relu')(input_signal)
encoded1 = Dense(M, activation='relu')(encoded)
encoded2 = Dense(2*n_channel, activation='linear')(encoded1)
encoded3 = Lambda(lambda x: np.sqrt(2*n_channel)*K.l2_normalize(x,axis=1))(encoded2)

encoded4 = GaussianNoise(np.sqrt(1/(2*R*Eb_N0)))(encoded3)

decoded = Dense(M, activation='relu')(encoded4)
decoded1 = Dense(M, activation='relu')(decoded)
decoded2 = Dense(M, activation='softmax')(decoded1)

autoencoder = Model(input_signal, decoded2)
adam = Adam(lr)
autoencoder.compile(optimizer=adam, loss='categorical_crossentropy')

In [9]:
print (autoencoder.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
lambda (Lambda)              (None, 16)                0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272   

In [10]:
autoencoder.fit(data, data,
                epochs=50,
                batch_size=2000)

Epoch 1/50
500/500 [==============================] - 2s 2ms/step - loss: 1.2044
Epoch 2/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0112
Epoch 3/50
500/500 [==============================] - 1s 2ms/step - loss: 0.0040
Epoch 4/50
500/500 [==============================] - 1s 2ms/step - loss: 0.0021
Epoch 5/50
500/500 [==============================] - 1s 2ms/step - loss: 0.0014
Epoch 6/50
500/500 [==============================] - 1s 3ms/step - loss: 0.0011
Epoch 7/50
500/500 [==============================] - 1s 2ms/step - loss: 8.2424e-04
Epoch 8/50
500/500 [==============================] - 1s 2ms/step - loss: 5.9488e-04
Epoch 9/50
500/500 [==============================] - 1s 3ms/step - loss: 5.2389e-04
Epoch 10/50
500/500 [==============================] - 1s 2ms/step - loss: 4.2775e-04
Epoch 11/50
500/500 [==============================] - 1s 3ms/step - loss: 4.3653e-04
Epoch 12/50
500/500 [==============================] - 1s 3ms/step - loss: 3.5679e-04
E

KeyboardInterrupt: 

In [11]:
from keras.models import load_model
#autoencoder.save('4_7_symbol_autoencoder_v_best.model')

In [12]:
#autoencoder_loaded = load_model('4_7_symbol_autoencoder_v_best.model')

In [13]:
encoder = Model(input_signal, encoded3)

In [14]:
# making decoder from full autoencoder
encoded_input = Input(shape=(2*n_channel,))

deco = autoencoder.layers[-3](encoded_input)
deco1 = autoencoder.layers[-2](deco)
deco2 = autoencoder.layers[-1](deco1)
decoder = Model(encoded_input, deco2)

In [15]:
N = 1000000
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [16]:
temp_test = 6
print (test_data[temp_test][test_label[temp_test]],test_label[temp_test])

1.0 6


In [17]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

# BER

Mean

nerrors/N

In [18]:
EbNodB_range = list(frange(0,11,1))
ber = [None]*len(EbNodB_range)
for i in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[i]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    noise = noise_std * np.random.randn(N,2*n_channel)
    encoded_signal = encoder.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder.predict(final_signal)
    ber[i] = (pred_final_signal.round()!=test_data).mean()
    print ('SNR:',EbNodB_range[i],'BER:',ber[i])

SNR: 0 BER: 0.0209948125
SNR: 1 BER: 0.0130291875
SNR: 2 BER: 0.0071254375


KeyboardInterrupt: 

### Matlab Array form

In [ ]:
for n in range(0,len(EbNodB_range)):
  print(ber[n], " ",end='')

0.021000375  0.0129495  0.006946625  0.0031653125  0.0011433125  0.0003190625  6.09375e-05  8.0625e-06  3.75e-07  1.25e-07  0.0  